# After years of thought it's finally here

# This is the code that will run at runtime; every elite run of the challenger

The preprocessing code only comes into effect when mons are added to or removed from the database entirely

In [206]:
import pandas as pd
import numpy as np
# import json  # apparently this doesn't need to be imported, pandas handles the pd.read_json

In [207]:
# This dictionary maps attacking types to their coverage
'''
with open(r"jsonfiles/Off.json","r") as p:
    Off=json.load(p)
print(Off)
'''
# Instead of a read/write operation, I just create the dictionary anew 3 cells later.

'\nwith open(r"jsonfiles/Off.json","r") as p:\n    Off=json.load(p)\nprint(Off)\n'

In [208]:
# This dictionary maps move names to their types

# the commented code is obsolete now that I shifted from saving just the dictionary as the json file, to saving the whole dataframe as a json file.
'''
with open(r"jsonfiles/move_dict.json","r") as p:
    move_dict=json.load(p)
print(move_dict)
'''
movetable = pd.read_json(r"jsonfiles/move_dict_table.json") 

In [209]:
movetable

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Acrobatics,Flying,110.0,Physical,NaN,None,NaN,NaN,NaN,None,None,None,NaN
Aerial Ace,Flying,60.0,Physical,NaN,None,NaN,NaN,NaN,Slicing,None,None,NaN
Air Slash,Flying,75.0,Special,NaN,None,NaN,NaN,NaN,Slicing,None,None,NaN
Alluring Voice,Fairy,80.0,Special,NaN,None,NaN,NaN,NaN,None,None,None,NaN
Aqua Cutter,Water,70.0,Physical,NaN,None,NaN,NaN,NaN,Slicing,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Charge,Electric,NaN,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN
Wood Hammer,Grass,NaN,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN
X Scissor,Bug,NaN,None,NaN,None,NaN,NaN,NaN,Slicing,None,None,NaN
Zap Cannon,Electric,NaN,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN


In [210]:
move_dict=movetable['Type'].to_dict()

In [211]:
# This dataframe has everything you'd need

core_df = pd.read_json(r"jsonfiles/core_df.json")
core_df.head()

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,None,132.0,132.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,None,150.0,115.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,None,70.0,100.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,None,105.0,60.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,None,140.0,60.0,None,RU,None,Yes,...,None,None,None,None,None,None,None,None,None,None


In [212]:
#Type Offense dictionary
#mirror coat and counter are typeless

# This has already been loaded as a json file.

Off={           
    'Normal':[],
    'Dragon':['Dragon'],
    'Ghost':['Psychic','Ghost'],
    'Electric':['Water','Flying'],
    'Fairy':['Fighting','Dragon','Dark'],
    'Poison':['Grass','Fairy'],

    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],

    'Fire':['Grass','Ice','Bug','Steel'],
    'Bug':['Grass','Psychic','Dark'],
    
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Freeze Dry':['Water','Grass','Ground','Flying','Dragon'],


    'Psychic':['Fighting','Poison'],
    'Flying':['Grass','Fighting','Bug'],
    'Steel':['Ice','Rock','Fairy'],

    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that

    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    
    'Dark':['Psychic','Ghost'],
    'Typeless':[],
    'nan' :[]

}

#notes: freeze dry is sneakily coded as an electric type move to include super effectiveness on water

In [213]:
core_df

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,None,132.0,132.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,None,150.0,115.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,None,70.0,100.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,None,105.0,60.0,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,None,140.0,60.0,None,RU,None,Yes,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,None,None,NaN,NaN,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Zangoose,Normal,None,None,None,NaN,NaN,None,RU,None,None,...,Payback,Pursuit,Power Trip,Counter,None,None,None,None,None,None
Zarude,Dark,Grass,None,None,NaN,NaN,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None
Zebstrika,Electric,None,None,None,NaN,NaN,None,RU,None,None,...,None,None,None,None,None,None,None,None,None,None


In [214]:
# This code has been copied from the other pokemon file

pre_coverage=core_df.drop(columns=['Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

coverage=pre_coverage.map(lambda x: Off.get(x))

elite_1='Grass'
elite_2='Ground'
elite_3='Ground'
elite_4='Ground'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
finally_filtered =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]
finally_filtered

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,NaN,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,NaN,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,Bug,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,NaN,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### The following code organizes the mons' base typing which is useful against the elite.

In [235]:
# We're processing the base typing separately from the moves because I want 3 columns reserved for the base typing.
# I don't want moves to mesh into the type columns and fill in the null spaces of the type columns.
# so I'm going to concatenate them later

useful_stab=finally_filtered.loc[:,['MonType1','MonType2','MonType3','Freeze_Dry_Stab']] # I want freeeze dry to appear as a stab type, not only when the elite has water, 
# but also when ice is super effective and an elite may bring an ice resist like gastrodon.
# Also even without a water elite, the champ often has a water mon so freeze dry is useful to show.
# This is why I have given freeze dry coverage over 5 types instead of just water.


# if the first type becomes nullified, I"m moving the second type to the spot of the first type. That's what the following code in this cell does.

''' #I'm not trying to force 3 columns anymore
useful_stab = useful_stab.apply(
    lambda row: row.dropna().reset_index(drop=True).reindex(range(3)),   # a bit different from the other code because I'm trying to force 3 columns instead of letting columns shrink too much
    axis=1,
)'''

useful_stab = useful_stab.apply(

    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,

)

useful_stab=useful_stab.rename(
    columns=lambda c: f'SupereffectiveStab{c + 1}'  #scalable no matter whether we've shrunk to 2 columns or 3 columns
)
#useful_stab.columns=['SupereffectiveStab1', 'SupereffectiveStab2', 'SupereffectiveStab3'] # 4 columns might be possible if a 3 column mon gets freeze dry
useful_stab.head(20)

,SupereffectiveStab1,SupereffectiveStab2,SupereffectiveStab3
Abomasnow,Grass,Ice,NaN
Accelgor,Bug,NaN,NaN
Aerodactyl,Flying,NaN,NaN
Aggron,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN
Altaria,Flying,NaN,NaN
Ambipom,NaN,NaN,NaN
Amoonguss,Grass,Poison,NaN
Ampharos,NaN,NaN,NaN
Araquanid,Water,Bug,NaN


In [220]:
# The stab score is how many montypes are good against the elite (how many montypes remained after the filtering and didn't get nullified.)
# the count method counts the number of non null elements.
useful_stab['StabScore']=useful_stab[[column for column in useful_stab.columns if 'SupereffectiveStab' in column]].count(axis='columns') # If I just pass in useful stab entirely instead of a list of 3 particular columns, then upon rerunning the cell it considers stabscore as a non null column. And that adds 1 to everything
#useful_stab['StabScore'].loc['Aurorus']

In [221]:
# Now that all the freeze dry stuff is done, we should drop that freeze dry column at the end.
# Because otherwise it will be considered in the next block of code
# and the result of this is that this freeze dry column is considered as a move column.
# so aurorus already has the move freeze dry, and it also has a fake move column which has the value freeze dry.
# this is why the freeze dry column should be dropped, so that it doesn't act like a fake move.

# okay now that I've moved this freeze dry column to the stab section, it won't interfere with the move columns. (Of the next code)
# so all the above comments are useless.


#finally_filtered=finally_filtered.drop(columns='Freeze_Dry_Stab')
# now this column automatically gets dropped because useful_stab has reshaped itself.

## The following code is to neaten the dataframe: to remove null values and slide all the relevant data left. Maybe it can be converted into a function.

This code also uses the usefu_stab group of columns and concatenates them with the random information columns and the move columns.
So useful stab 

In [222]:
df_where_NaNs_are_removable=finally_filtered.loc[:,'Move1':] # this dataframe ignores the stab type, and has the Moves.

df_where_NaNs_are_removable = df_where_NaNs_are_removable.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

df_where_NaNs_are_removable = df_where_NaNs_are_removable.rename(
    columns=lambda c: f'Move{c + 1}'
)
df_where_NaNs_are_removable

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15
Abomasnow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Sludge Bomb,Venoshock,Water Shuriken,Energy Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Aqua Tail,Fire Fang,Flamethrower,Ice Fang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Aqua Tail,Surf,Fire Punch,Flamethrower,Avalanche,Ice Punch,Ice Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,Energy Ball,Giga Drain,Solar Beam,Mystical Fire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,Grass Knot,Solar Beam,Giga Drain,Heat Wave,Signal Beam,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,Solar Beam,Giga Drain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Gunk Shot,Poison Jab,Surf,Grass Knot,Solar Beam,Seed Bomb,Giga Drain,Flamethrower,Fire Punch,X Scissor,Ice Beam,Ice Punch,NaN,NaN,NaN
Zarude,Uturn,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


have a filter to avoid inconsistent moves like fling, pursuit
maybe bulldoze and rock tomb too, although those moves could come under another filter

In [223]:
#final=pd.concat([finally_filtered.loc[:,:'Trash'],useful_stab,df_where_NaNs_are_removable], axis=1)

#everything_before_move1=finally_filtered.loc[:,:finally_filtered.columns[finally_filtered.columns.get_loc('Move1')-1]]
# I just realized that for this simple thing I could have just done iloc. But it appears iloc shouldn't have the -1.
everything_before_move1=finally_filtered.iloc[:,:finally_filtered.columns.get_loc('Move1')]

# finding the index of Move1, and subtracting 1 gives us the previous column index. inputting that into finally_filtered.columns gives us the column name.
# refer the tosplit, part_1, part_2 stuff around SrijanSort (from preprocessing) to understand the above line, and why I stopped using 'Trash' as the endpoint of slicing.


final=pd.concat([everything_before_move1,useful_stab,df_where_NaNs_are_removable], axis=1)
final

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,NaN,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ice Punch,Ice Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,NaN,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,Bug,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Seed Bomb,Giga Drain,Flamethrower,Fire Punch,X Scissor,Ice Beam,Ice Punch,NaN,NaN,NaN
Zarude,NaN,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
# I still need to add all those wishlist, used, tier values back
final[['Tier','In Wishlist','Used','Trash']]=core_df[['Tier','In Wishlist','Used','Trash']]
# final did have these 4 column names (in the right spots too), but they were initially null (due to .map() stuff long ago). 
# With this code, I set all the values to the values found in core_df.

In [ ]:
# I've realized I also want to be able to see the full typing of the mons even if the full typing isn't useful offensively
# So at one side of the table I'll have the full typing, and at the other side I'll have the super effective typing
# so I'm directly copying the original typing too

final[['MonType1','MonType2','MonType3']]=core_df[['MonType1','MonType2','MonType3']]

# The Freeze dry column doesn't get added here because it's unnecessary. I don't want to look at it everytime alongside ice when both are non supereffective.

I'm kinda cooked when it comes to megas. When the megas are also RU, for now, I'll give the mons the stats of their megas instead of their regular stats.
I'll also lump all abilities together, mega or not.

In [226]:
final[['Atk','SpA','Abilities']]=core_df[['Atk','SpA','Abilities']]

Code to make it sortable by stab

In [227]:
final=final.sort_values(by='StabScore', kind = 'stable', ascending=False) # The kind parameter is needed to keep the sorting stable.

In [228]:
final=final.drop(columns='StabScore') # Now that it's sorted, we no longer need this StabScore

In [229]:
# A few stuff I don't need for now, might change later
final=final.drop(columns=['Tier','In Wishlist', 'Trash'])

In [230]:
final.to_csv('Best_Mons_at_runtime_Versus_The_Elite.csv')

copy paste technician mons before and after all the move deletions so that they don't lose their moves
Also Strong Jaw mons and stuff ig

In [231]:
clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=np.NaN)

# this was the cell I used to make certain cells null

NameError: name 'clean_df' is not defined